# NYC Yellow Cab Fare Prediction

### Dataset link: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page 

We selected the July data for Yellow Cab and convert parquet file to csv file for convenience.

## Data Preprocessing Steps

In [37]:
from pyspark.sql import SparkSession

In [38]:
#Create a Spark instance and pre-allocate Max memory to prevent errors.
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("taxi-duration-prediction-2")\
            .config("spark.executor.memory", MAX_MEMORY)\
            .config("spark.driver.memory", MAX_MEMORY).getOrCreate()

In [39]:
trip_files = "/Users/kyungminpark/Desktop/Fall2023/CS4641/data/trips/*"

In [40]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header=True)

In [58]:
from pyspark.sql.functions import col

# "Cast the "total_amount column, trip_distance, passenger_count" to an integer type 
# since the original type was string and we need to transform to double for concatenation later
trips_df = trips_df.withColumn("total_amount", col("total_amount").cast("double"))
trips_df = trips_df.withColumn("trip_distance", col("trip_distance").cast("double"))
trips_df = trips_df.withColumn("passenger_count", col("passenger_count").cast("double"))


In [59]:
#Check the Schema
trips_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)



In [60]:
## Transformed for use in SQL: "CAST the # total_amount column to an integer data type.
trips_df.createOrReplaceTempView("trips")

In [61]:
# PULocationID - pick up location ID
# DOLocationID - drop off location ID


query = """
SELECT
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    (SELECT
        *,
        TO_DATE(t.tpep_pickup_datetime) AS pickup_date
    FROM
        trips t)
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND pickup_date >= '2023-01-01'
    AND pickup_date < '2023-08-01'
"""
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [62]:
data_df.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+
|            1.0|               161|                141|         0.97|          0|     Sunday|        14.3|
|            1.0|                43|                237|          1.1|          0|     Sunday|        16.9|
|            1.0|                48|                238|         2.51|          0|     Sunday|        34.9|
|            0.0|               138|                  7|          1.9|          0|     Sunday|       20.85|
|            1.0|               107|                 79|         1.43|          0|     Sunday|       19.68|
|            1.0|               161|                137|         1.84|          0|     Sunday|        27.8|
|            1.0|           

In [63]:
# Separate train set and test set
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)

In [64]:
# Before reading a data, transform the type of data
data_dir = "/Users/kyungminpark/Desktop/Fall2023/CS4641/data/"

In [66]:
train_df.write.format("parquet").save(f"{data_dir}/train/")
test_df.write.format("parquet").save(f"{data_dir}/test/")

23/11/07 08:33:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , , , , 
 Schema: tpep_pickup_datetime, passenger_count, trip_distance, PULocationID, DOLocationID, total_amount
Expected: tpep_pickup_datetime but found: 
CSV file: file:///Users/kyungminpark/Desktop/Fall2023/CS4641/data/trips/taxi_zones.shp
23/11/07 08:33:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , , , , 
 Schema: tpep_pickup_datetime, passenger_count, trip_distance, PULocationID, DOLocationID, total_amount
Expected: tpep_pickup_datetime but found: 
CSV file: file:///Users/kyungminpark/Desktop/Fall2023/CS4641/data/trips/taxi_zones.dbf
23/11/07 08:33:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , , , , , 
 Schema: tpep_pickup_datetime, passenger_count, trip_distance, PULocationID, DOLocationID, total_amount
Expected: tpep_pickup_datetime but found: 
CSV file: file:///Users/kyungminpark/Desktop/Fall2023/CS4641/data/trips/t

In [67]:
# Using the schema, read the Parquet files, now we can use train_df and test_df anytime because we saved.
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

In [68]:
train_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: string (nullable = true)
 |-- dropoff_location_id: string (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [69]:
## Categorical Feature PreProcessing Steps
from pyspark.ml.feature import OneHotEncoder, StringIndexer

cat_feats = [ ##categorical features
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

## Data flows through the pipeline stages (the pipeline consists of multiple stages, and each stage is executed sequentially)
stages = []

for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c, outputCol= c + "_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c + "_onehot"])
    stages += [cat_indexer, onehot_encoder]

In [70]:
stages

[StringIndexer_486108410052,
 OneHotEncoder_21e3f664c40f,
 StringIndexer_94e6e8c43ee6,
 OneHotEncoder_281d6a9bad7b,
 StringIndexer_203c72cfb6b2,
 OneHotEncoder_801b9a77e3b9]

In [71]:
## Numerical Feature PreProcessing Steps

from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "pickup_time",
    "passenger_count",
    "trip_distance"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol= n + "_vecotr")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol= n + "_scaled")
    stages += [num_assembler, num_scaler]

In [72]:
## Concatenate Categoriacal Feature and Numerical Feature

assembler_inputs = [c + "_onehot" for c in cat_feats] + [n + "_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="feature_vector")
stages += [assembler]

In [73]:
#Build the Pipeline
from pyspark.ml import Pipeline

transform_stages = stages

# Create the pipeline with the defined transformation stages
pipeline = Pipeline(stages=transform_stages)
## Fit the pipeline on the training data frame to create a fitted transformer
fitted_transformer = pipeline.fit(train_df)

In [74]:
vtrain_df = fitted_transformer.transform(train_df)

In [75]:
vtrain_df.printSchema()

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: string (nullable = true)
 |-- dropoff_location_id: string (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- pickup_time_vecotr: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- passenger_count_vecotr: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vecotr: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- feature_vector: vector (nullab

In [76]:
# We used one of the "Supervised Learning " which is LinearRegression
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    solver="normal",
    labelCol="total_amount", ##총 택시비
    featuresCol="feature_vector"
)

In [77]:
model = lr.fit(vtrain_df)

23/11/07 08:33:39 WARN Instrumentation: [e7e7bcf6] regParam is zero, which might cause numerical instability and overfitting.
23/11/07 08:33:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/07 08:33:41 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/11/07 08:33:42 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
23/11/07 08:33:42 WARN Instrumentation: [e7e7bcf6] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [78]:
## Create the test data frame
vtest_df = fitted_transformer.transform(test_df)

In [79]:
predictions = model.transform(vtest_df)

In [80]:
predictions.cache()

DataFrame[passenger_count: double, pickup_location_id: string, dropoff_location_id: string, trip_distance: double, pickup_time: int, day_of_week: string, total_amount: double, pickup_location_id_idx: double, pickup_location_id_onehot: vector, dropoff_location_id_idx: double, dropoff_location_id_onehot: vector, day_of_week_idx: double, day_of_week_onehot: vector, pickup_time_vecotr: vector, pickup_time_scaled: vector, passenger_count_vecotr: vector, passenger_count_scaled: vector, trip_distance_vecotr: vector, trip_distance_scaled: vector, feature_vector: vector, prediction: double]

In [81]:
predictions.select(["trip_distance", "day_of_week", "total_amount", "prediction"]).show()

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|          0.3|    Tuesday|        12.6|14.740826847804103|
|          1.5|   Saturday|        14.0|17.954214720138758|
|          1.9|   Thursday|       21.35|20.182057693408087|
|          2.3|   Thursday|       29.85|23.614430372250908|
|         16.7|     Friday|       90.55| 87.12648919285517|
|          0.9|    Tuesday|        17.0|15.915257217012602|
|          1.9|     Sunday|        22.4|19.520983350048976|
|          2.6|  Wednesday|        27.9|24.495638038688984|
|          3.3|    Tuesday|        47.0|27.323111652093385|
|          3.3|     Monday|        29.7|26.673399221992423|
|          2.3|  Wednesday|        33.2|23.425178359618254|
|          0.6|  Wednesday|        15.4|16.090424087444426|
|          0.8|  Wednesday|       14.25| 17.24093745398843|
|          0.9|   Thursday|       14.25|

This table is indicating the Yellow Cab fare prediction(Yellow Cab 2023 July data) for total_amount. Pay close attention to the two columns, total_amount, and predictions

In [82]:
model.summary.rootMeanSquaredError

6.671272023422926

In [83]:
model.summary.r2

0.9105461273892894

As we can see the above result, the accuracy of prediction of total_amount is around 91%.